In [1]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores.cassandra import Cassandra
from dotenv import load_dotenv
import cassio
load_dotenv()


USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\Data Science\LangChain\GenAi_BotGemma\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
groq_api_key=os.environ['GROQ_API_KEY']
llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")
ASTRA_DB_KEYSPACE=os.getenv("ASTRA_DB_KEYSPACE")  

cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID,keyspace=ASTRA_DB_KEYSPACE)

In [11]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader=WebBaseLoader(web_path=('https://lilianweng.github.io/posts/2023-06-23-agent/'),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-title","post-content","post-header"))))

text_document=loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs=text_splitter.split_documents(text_document)
docs[:5]

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refi

In [5]:
# convert data to  vec and store in astra DB

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001",google_api_key=google_api_key)

## DB connectoin
astra_vector_store=Cassandra(embedding=embeddings,table_name="groq_demo",session=None,keyspace=ASTRA_DB_KEYSPACE)

In [12]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
astra_vector_store.add_documents(docs)
print("Inserted %i headlines" % len(docs))
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 63 headlines


In [13]:
from langchain_core.prompts import ChatPromptTemplate
#llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it")
prompt=ChatPromptTemplate.from_template("""
Anser the question based on the provided content 
<context> {context} </context>
Question: {input}
""")


In [14]:
astra_vector_index.query("Chiain of thoughts",llm=llm)

"This document describes how LLMs can be used to build autonomous agents.  It talks about several techniques for improving the performance of these agents, such as:\n\n* **Chain of Thought (CoT):**  Breaking down complex tasks into smaller, simpler steps.\n* **Tree of Thoughts (ToT):**  Exploring multiple reasoning possibilities at each step.\n* **Chain of Hindsight (CoH):**  Using past outputs and feedback to improve future outputs.\n\n\nHowever,  it doesn't explain how to build an agent that can learn to play chess. \n"

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever=astra_vector_store.as_retriever()
document_chain=create_stuff_documents_chain(llm,prompt)
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [16]:
response=retrieval_chain.invoke({"input":"What is Chain of thoughts"})
response

{'input': 'What is Chain of thoughts',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Chain of Hindsight (CoH; Liu et al. 2023) encourages the model to improve on its own outputs by explicitly presenting it with a sequence of past outputs, each annotated with feedback. Human feedback data is a collection of $D_h = \\{(x, y_i , r_i , z_i)\\}_{i=1}^n$, where $x$ is the prompt, each $y_i$ is a model completion, $r_i$ is the human rating of $y_i$, and $z_i$ is the corresponding human-provided hindsight feedback. Assume the feedback tuples are ranked by reward, $r_n \\geq r_{n-1} \\geq \\dots \\geq r_1$ The process is supervised fine-tuning where the data is a sequence in the form of $\\tau_h = (x, z_i, y_i, z_j, y_j, \\dots, z_n, y_n)$, where $\\leq i \\leq j \\leq n$. The model is finetuned to only predict $y_n$ where conditioned on the sequence prefix, such that the model can self-reflect to produce better output based on 